In [ ]:
import pandas as pd
import numpy as np
primary_schools = pd.read_csv('total_schools_2030.csv')

primary_schools = primary_schools[['latitude','longitude']]

In [ ]:
primary_schools = primary_schools.rename(
    columns = {
        'latitude':'lat',
        'longitude':'lon'
    }
)

primary_schools['school_index'] = np.arange(0,len(primary_schools))
primary_schools['max_children'] = 620
primary_schools['q_children'] = 0

print(primary_schools.head())


In [ ]:
density_pop = pd.read_csv('density2020school.csv',names=['lon','lat','densidad'], header=0)
print(sum(density_pop['densidad']) / len(primary_schools))

In [ ]:
density_pop['densidad'] = np.ceil(density_pop['densidad'])
density_pop['group'] = np.arange(1,len(density_pop)+1)


In [ ]:
# density_pop = density_pop.loc[density_pop.index.repeat(density_pop['densidad'])]

In [ ]:
primary_schools['q_children'].sum()


In [ ]:
from scipy.spatial import cKDTree
from tqdm import tqdm
import numpy as np

# Crear un KD-Tree con las ubicaciones de las escuelas
tree_schools = cKDTree(primary_schools[['lat', 'lon']].values)

total_items = len(density_pop)

# Crea una barra de progreso para el bucle
for i in tqdm(range(total_items)):
    child_location = density_pop.iloc[i][['lat', 'lon']].values

    # Buscar las escuelas más cercanas a la ubicación del niño
    # y obtener los índices de las escuelas
    indices_school = tree_schools.query(child_location, k=500)[1]

    found = False

    for j in indices_school:
        if primary_schools.iloc[j]['q_children'] < primary_schools.iloc[j]['max_children']:
            if density_pop.at[i, 'densidad'] > (primary_schools.iloc[j]['max_children'] - primary_schools.iloc[j]['q_children']):
                excess_children = density_pop.at[i, 'densidad'] - (primary_schools.iloc[j]['max_children'] - primary_schools.iloc[j]['q_children'])
                primary_schools.at[j, 'q_children'] = primary_schools.iloc[j]['max_children']
                density_pop.at[i, 'densidad'] = excess_children
            else:
                primary_schools.at[j, 'q_children'] += density_pop.at[i, 'densidad']
                density_pop.at[i, 'densidad'] = 0
                
            density_pop.at[i, 'school_index'] = j
            found = True
            break

    if not found:
        # Handle the case when no school with available capacity is found
        print('No se encontró una escuela con capacidad para el niño ' + str(i))


In [ ]:
primary_schools['q_children'].sum()

In [ ]:
table = density_pop.groupby('school_index').count().sort_values(by = 'densidad')
print(table)


print(primary_schools)



In [ ]:
primary_schools.to_csv('primary_schools2030.csv')
density_pop.to_csv('density_prop.csv')